In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

def plot_orderbook_with_arbitrage(df: pd.DataFrame, symbol: str, save_dir: str = "arb_charts_interactive"):
    df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
    exchanges = df["exchange"].unique()
    color_map = px.colors.qualitative.Plotly

    fig = go.Figure()

    # 📈 绘制 bid1 和 ask1
    for i, ex in enumerate(exchanges):
        sub = df[df["exchange"] == ex]
        fig.add_trace(go.Scatter(
            x=sub["datetime"], y=sub["bid1_price"],
            mode="lines", name=f"{ex} bid1",
            line=dict(color=color_map[i % len(color_map)], dash="dash"), opacity=0.7
        ))
        fig.add_trace(go.Scatter(
            x=sub["datetime"], y=sub["ask1_price"],
            mode="lines", name=f"{ex} ask1",
            line=dict(color=color_map[i % len(color_map)], dash="solid"), opacity=0.7
        ))

    # 📉 套利计算
    arbs = []
    for time, group in df.groupby("datetime"):
        bid_row = group.loc[group["bid1_price"].idxmax()]
        ask_row = group.loc[group["ask1_price"].idxmin()]
        spread = bid_row["bid1_price"] - ask_row["ask1_price"]
        if spread > 0:
            pct = (spread / ask_row["ask1_price"]) * 100
            arbs.append({
                "time": time, "spread_pct": pct,
                "buy": ask_row["exchange"], "sell": bid_row["exchange"]
            })

    arbs_df = pd.DataFrame(arbs)
    fig.add_trace(go.Scatter(
        x=arbs_df["time"], y=arbs_df["spread_pct"],
        mode="lines", name="Arbitrage %",
        line=dict(color="purple", width=2)
    ))

    # 📍 最大套利点
    if not arbs_df.empty:
        max_row = arbs_df.loc[arbs_df["spread_pct"].idxmax()]
        fig.add_trace(go.Scatter(
            x=[max_row["time"]], y=[max_row["spread_pct"]],
            mode="markers+text", name="Max Arbitrage",
            text=[f"{max_row['spread_pct']:.4f}%\nBuy:{max_row['buy']}\nSell:{max_row['sell']}"],
            marker=dict(color="red", size=10), textposition="top center"
        ))

    fig.update_layout(
        title=f"{symbol} Orderbook & Arbitrage",
        xaxis_title="Time", yaxis_title="Price / Spread %",
        hovermode="x unified", legend=dict(orientation="h")
    )

    os.makedirs(save_dir, exist_ok=True)
    html_path = os.path.join(save_dir, f"{symbol}.html")
    fig.write_html(html_path)
    print(f"✅ 可交互图已保存：{html_path}")

# 📂 主流程：批量处理所有 CSV
csv_dir = "../csv_orderbooks_symbol"
for fname in os.listdir(csv_dir):
    if fname.endswith(".csv"):
        symbol = os.path.splitext(fname)[0]
        try:
            df = pd.read_csv(os.path.join(csv_dir, fname))
            plot_orderbook_with_arbitrage(df, symbol)
        except Exception as e:
            print(f"⚠️ 处理 {fname} 时出错：{e}")


✅ 可交互图已保存：arb_charts_interactive/ob_AERGO_USDT_USDT.html
✅ 可交互图已保存：arb_charts_interactive/ob_1INCH_USDT_USDT.html
✅ 可交互图已保存：arb_charts_interactive/ob_ACH_USDT_USDT.html
✅ 可交互图已保存：arb_charts_interactive/ob_AEVO_USDT_USDT.html


KeyboardInterrupt: 